In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
from IPython.display import clear_output
import face_recognition
import argparse
import dlib
import os
from utils.aux_functions import *
import sys
import math
import sys
import datetime
from RetinaFace.retinaface_cov import RetinaFaceCoV
from PIL import Image, ImageFile

%matplotlib inline

In [5]:
### MaskTheFace Argument load & Generate Masked Image from Selected Folders ###

parser = argparse.ArgumentParser(
    description="MaskTheFace - Python code to mask faces dataset"
)

parser.add_argument(
    "--path",
    type=str,
    default="../users",
    help="Path to either the folder containing images or the image itself",
)
parser.add_argument(
    "--mask_type",
    type=str,
    default="KN95",
    choices=["surgical", "N95", "KN95", "cloth", "gas", "inpaint", "random", "all"],
    help="Type of the mask to be applied. Available options: all, surgical_blue, surgical_green, N95, cloth",
)

parser.add_argument(
    "--pattern",
    type=str,
    default="",
    help="Type of the pattern. Available options in masks/textures",
)

parser.add_argument(
    "--pattern_weight",
    type=float,
    default=0.5,
    help="Weight of the pattern. Must be between 0 and 1",
)

parser.add_argument(
    "--color",
    type=str,
    default="#0473e2",
    help="Hex color value that need to be overlayed to the mask",
)

parser.add_argument(
    "--color_weight",
    type=float,
    default=0.5,
    help="Weight of the color intensity. Must be between 0 and 1",
)

parser.add_argument(
    "--code",
    type=str,
    default="",
    help="Generate specific formats",
)

parser.add_argument(
    "--verbose", dest="verbose", action="store_true", help="Turn verbosity on"
)

parser.add_argument(
    "--write_original_image",
    dest="write_original_image",
    action="store_true",
    help="If true, original image is also stored in the masked folder",
)

parser.set_defaults()



args = parser.parse_args(args=[])
args.write_path = args.path

# Set up dlib face detector and predictor
args.detector = dlib.get_frontal_face_detector()
path_to_dlib_model = "dlib_models/shape_predictor_68_face_landmarks.dat"
if not os.path.exists(path_to_dlib_model):
    download_dlib_model()

args.predictor = dlib.shape_predictor(path_to_dlib_model)

# Extract data from code
mask_code = "".join(args.code.split()).split(",")
args.code_count = np.zeros(len(mask_code))
args.mask_dict_of_dict = {}

for i, entry in enumerate(mask_code):
    mask_dict = {}
    mask_color = ""
    mask_texture = ""
    mask_type = entry.split("-")[0]
    if len(entry.split("-")) == 2:
        mask_variation = entry.split("-")[1]
        if "#" in mask_variation:
            mask_color = mask_variation
        else:
            mask_texture = mask_variation
    mask_dict["type"] = mask_type
    mask_dict["color"] = mask_color
    mask_dict["texture"] = mask_texture
    args.mask_dict_of_dict[i] = mask_dict

# Check if path is file or directory or none
is_directory, is_file, is_other = check_path(args.path)
display_MaskTheFace()

if is_directory:
    path, dirs, files = os.walk(args.path).__next__()
    file_count = len(files)
    dirs_count = len(dirs)
    if len(files) > 0:
        print_orderly("Masking image files", 60)

    # Process files in the directory if any
    for f in tqdm(files):
        image_path = path + "/" + f

        write_path = path
        if not os.path.isdir(write_path):
            os.makedirs(write_path)

        if is_image(image_path):
            # Proceed if file is image
            if args.verbose:
                str_p = "Processing: " + image_path
                tqdm.write(str_p)

            split_path = f.rsplit(".")
            masked_image, mask, mask_binary_array, original_image = mask_image(
                image_path, args
            )
            for i in range(len(mask)):
                w_path = (
                    write_path
                    + "/"
                    + split_path[0]
                    + "_"
                    + mask[i]
                    + "."
                    + split_path[1]
                )
                img = masked_image[i]
                cv2.imwrite(w_path, img)

    print_orderly("Masking image directories", 60)

    # Process directories withing the path provided
    for d in tqdm(dirs):
        dir_path = args.path + "/" + d
        dir_write_path = args.write_path + "/" + d
        if not os.path.isdir(dir_write_path):
            os.makedirs(dir_write_path)
        _, _, files = os.walk(dir_path).__next__()

        # Process each files within subdirectory
        for f in files:
            image_path = dir_path + "/" + f
            if args.verbose:
                str_p = "Processing: " + image_path
                tqdm.write(str_p)
            write_path = dir_write_path
            if is_image(image_path):
                # Proceed if file is image
                split_path = f.rsplit(".")
                masked_image, mask, mask_binary, original_image = mask_image(
                    image_path, args
                )
                for i in range(len(mask)):
                    w_path = (
                        write_path
                        + "/"
                        + split_path[0]
                        + "_"
                        + mask[i]
                        + "."
                        + split_path[1]
                    )
                    w_path_original = write_path + "/" + f
                    img = masked_image[i]
                    # Write the masked image
                    cv2.imwrite(w_path, img)
                    if args.write_original_image:
                        # Write the original image
                        cv2.imwrite(w_path_original, original_image)

            if args.verbose:
                print(args.code_count)

# Process if the path was a file
elif is_file:
    print("Masking image file")
    image_path = args.path
    write_path = args.path.rsplit(".")[0]
    if is_image(image_path):
        # Proceed if file is image
        # masked_images, mask, mask_binary_array, original_image
        masked_image, mask, mask_binary_array, original_image = mask_image(
            image_path, args
        )
        for i in range(len(mask)):
            w_path = write_path + "_" + mask[i] + "." + args.path.rsplit(".")[1]
            img = masked_image[i]
            cv2.imwrite(w_path, img)
else:
    print("Path is neither a valid file or a valid directory")
print("Processing Done")

  0%|          | 0/4 [00:00<?, ?it/s]

 __  __           _ _______ _          ______
|  \/  |         | |__   __| |        |  ____|
| \  / | __ _ ___| | _| |  | |__   ___| |__ __ _  ___ ___
| |\/| |/ _` / __| |/ / |  | '_ \ / _ \  __/ _` |/ __/ _ \
| |  | | (_| \__ \   <| |  | | | |  __/ | | (_| | (_|  __/
|_|  |_|\__,_|___/_|\_\_|  |_| |_|\___|_|  \__,_|\___\___|
-------------------------------------------------------------
-------------------- Masking image files --------------------
-------------------------------------------------------------


100%|██████████| 1/1 [00:00<00:00, 671.41it/s]

-------------------------------------------------------------
----------------- Masking image directories -----------------
-------------------------------------------------------------
Processing Done


In [ ]:
### RetinaFace Covid Load and Find Face Bounding Box
thresh = 0.8
mask_thresh = 0.2
scales = [640, 1080]

count = 1

gpuid = 0
detector = RetinaFaceCoV('./models/cov2/mnet_cov2', 0, gpuid, 'net3l')

In [ ]:



img = cv2.imread('../users/hyungwoo_KN95.jpg')
print(img.shape)
im_shape = img.shape
target_size = scales[0]
max_size = scales[1]
im_size_min = np.min(im_shape[0:2])
im_size_max = np.max(im_shape[0:2])
#im_scale = 1.0
#if im_size_min>target_size or im_size_max>max_size:
im_scale = float(target_size) / float(im_size_min)
# prevent bigger axis from being more than max_size:
if np.round(im_scale * im_size_max) > max_size:
    im_scale = float(max_size) / float(im_size_max)

print('im_scale', im_scale)

scales = [im_scale]
flip = False

for c in range(count):
      faces, landmarks = detector.detect(img, thresh, scales=scales, do_flip=flip)

if faces is not None:
    print('find', faces.shape[0], 'faces')
    for i in range(faces.shape[0]):
        #print('score', faces[i][4])
        face = faces[i]
        box = face[0:4].astype(np.int)
        mask = face[5]
        print(i,box,mask)
        #color = (255,0,0)
        if mask>=mask_thresh:
            color = (0,0,255)
        else:
            color = (0,255,0)
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), color, 2)
    landmark5 = landmarks[i].astype(np.int)
    #print(landmark.shape)
    for l in range(landmark5.shape[0]):
        color = (255,0,0)
        cv2.circle(img, (landmark5[l][0], landmark5[l][1]), 1, color, 2)
    filename = './cov_test2.jpg'
    print('writing', filename)
    cv2.imwrite(filename, img)



In [3]:
### DATABASE에 저장된 각 이미지들에서 face embedding을 추출하여 저장해놓음
### MaskTheFace를 통해 생성해놓은 이미지 또한 detection 및 feature extraction을 진행해야하므로 retinaface와 같은
### detection with mask 모델들의 도입이 필요함
### 추가적으로 face_recognition의 face embedding 결과가 동양인에 대해서는 다소 미흡한 부분이 있음으로 확인되어 다른 모델 탐구 필요


known_face_encodings = []
known_face_names = []

files = glob.glob('../users/*.jpg')

print(files)

for file in files:
    if face_recognition.face_encodings(cv2.imread(file)):
        known_face_encodings.append(face_recognition.face_encodings(cv2.imread(file))[0])
        known_face_names.append(file[6:-4])
        print(file[6:-4])

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

print(known_face_encodings, known_face_names)

['../users/hyungwoo.jpg', '../users/sangrae.jpg', '../users/biden.jpg', '../users/obama.jpg']
rs/hyungwoo
rs/sangrae
rs/biden
rs/obama
[array([ 1.19246077e-02,  8.35554376e-02,  5.24901077e-02, -2.26907805e-03,
       -7.17738122e-02, -3.65886763e-02, -4.70025614e-02, -7.38082528e-02,
        9.71959382e-02, -1.00572616e-01,  1.95351794e-01, -7.06306174e-02,
       -1.92481488e-01, -6.44390509e-02,  1.88723207e-05,  1.30522430e-01,
       -1.37330458e-01, -1.32039890e-01, -4.61875275e-02, -9.54853743e-03,
        8.99488851e-02,  3.56770754e-02,  6.58243522e-03,  6.60214946e-03,
       -1.14531048e-01, -3.26654583e-01, -1.31850943e-01, -9.64238569e-02,
        1.29371947e-02, -9.05379504e-02, -7.47981146e-02,  3.87385115e-02,
       -1.19325981e-01, -2.23364867e-02,  5.97192682e-02,  7.73799345e-02,
       -4.38191779e-02, -6.03248030e-02,  2.09903911e-01,  2.78726965e-03,
       -2.40433067e-01,  8.61801654e-02,  5.63548878e-02,  2.55913049e-01,
        2.47723162e-01,  4.35683206e-02

In [10]:
video = cv2.VideoCapture('http://posproject201013.iptime.org:8787/video_feed')

while True:
    _, frame = video.read()

    origin = frame
        # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_small_frame, model='cnn')
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        
        name = "Unknown"

        # Or instead, use the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        
        print(len(face_distances), best_match_index, len(matches))
        print(face_distances, face_distances.shape)
        print(best_match_index)
        print(len(matches))
        
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        face_names.append(name)

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        
    masked_image, mask, mask_binary_array, original_image = mask_image_file(origin, args)
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.axis('off')
    
   
    if len(masked_image):
        frame = cv2.cvtColor(masked_image[0], cv2.COLOR_BGR2RGB)
        plt.imshow(frame)
    else:
        plt.imshow(frame)
    
    plt.show()
    cv2.waitKey(1)
    clear_output(wait=True)

4 1 4
[0.59140878 0.56474993 0.8543517  0.72604811] (4,)
1
4


KeyboardInterrupt: 